# [D&A Conexão Digital] Classificador de documentos para concessão de benefícios

In [ ]:
!pip install pdfplumber PyPDF2 pdf2image pytesseract yake language_tool_python pyspellchecker
!sudo apt install tesseract-ocr
!apt-get install tesseract-ocr-por #tesseract em pt-br (por)
!apt-get install -y poppler-utils #necessária para conveter pdf -> imagem
!pip install Pillow==9.5.0 #deve-se utilizar essa versão para: pdf -> imagem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.8 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 1s (5,202 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fro

In [ ]:
#lista de idiomas do tesseract
!tesseract --list-langs

List of available languages (3):
eng
osd
por


In [ ]:
!mkdir tessdata
!wget -O ./tessdata/por.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata?raw=true
config_tesseract = '--tessdata-dir tessdata'

--2023-06-09 12:56:12--  https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/tesseract-ocr/tessdata/raw/main/por.traineddata [following]
--2023-06-09 12:56:12--  https://github.com/tesseract-ocr/tessdata/raw/main/por.traineddata
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tesseract-ocr/tessdata/main/por.traineddata [following]
--2023-06-09 12:56:12--  https://raw.githubusercontent.com/tesseract-ocr/tessdata/main/por.traineddata
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awa

In [ ]:
import re
import pandas as pd
import os, sys
import numpy as np

import os
from pdf2image import convert_from_path
from PIL import Image
import pdfplumber
import cv2 #opencv
from collections import Counter
import pytesseract

import glob

from yake import KeywordExtractor
from spellchecker import SpellChecker
import language_tool_python

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

#import matplotlib.pyplot as plt foi utilizando apenas para testes

In [ ]:
print(pytesseract.get_tesseract_version())

4.1.1


# Funções para o output do OCR

In [ ]:
def padrao_nome(texto):
    regex = r'(?i)(?:NOME|Nome da Pessoa Física|COMPLETO|NAME|NOMBRE)[:\s]*([^\n,]+)'
    resultado = re.search(regex, texto, re.IGNORECASE)
    return resultado.group(1) if resultado else None

def fantasia_nome(texto):
    regex = r'(?i)(?:NOME EMPRESARIAL|NOME FANTASIA| Nome de Fantansia)[:\s]*([^\n,]+)'
    resultado = re.search(regex, texto, re.IGNORECASE)
    return resultado.group(1) if resultado else None

def padrao_cpf(texto):
    regex = r'CPF:\s*(\d{3}\.\d{3}\.\d{3}-\d{2})'
    resultado = re.search(regex, texto, re.IGNORECASE)
    return resultado.group(1) if resultado else None

def padrao_cnpj(texto):
    regex = r'CNPJ:\s*(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})'
    resultado = re.search(regex, texto, re.IGNORECASE)
    return resultado.group(1) if resultado else None

def padrao_situacao_cadastral(texto):
    regex = r'(?:Situação Cadastral:)\s*(.*)\n'
    resultado = re.search(regex, texto, re.IGNORECASE)
    return resultado.group(1) if resultado else None

def padrao_data_nascimento(texto):
    regex = r'(?:Data de Nascimento:)\s*(.*)\n'
    resultado = re.search(regex, texto, re.IGNORECASE)
    return resultado.group(1) if resultado else None

import re

def find_uf(texto):
    uf_list = ["AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"]
    for uf in uf_list:
        if re.search(r'\b' + uf + r'\b', texto):
            return uf
    return None

def buscar_siglas(nome_arquivo):
    siglas = ['CCIR', 'CNPJ', 'CPF', 'CTRU', 'IE', 'IRR', 'ITR', 'NIRF',
              'PRMA', 'PRONAF', 'RG', 'RURL', 'VIDA', 'RMMV', 'OUTA',
              'LIAM', 'AUTODECLARAÇÃO', 'ENBE', 'NB', 'NIS', 'PRIC',
              'PRIE', 'ITR', 'CONS']

    for sigla in siglas:
        if sigla in nome_arquivo:
            return sigla

    return None

def atribuir_distribuidora(nome_arquivo):
    if nome_arquivo.startswith('1'):
        return 'NCLB'
    elif nome_arquivo.startswith('2'):
        return 'NPER'
    elif nome_arquivo.startswith('3'):
        return 'NCSR'
    else:
        return 'None'

# Converter de .pdf para .jpeg e aplicar o OCR

In [ ]:
# Define o caminho do arquivo PDF.
pdf_path = r"/content/sample_data/RG_HENRIQUE.pdf"

def convert_pdf_to_image(pdf_path):
    # Converte pdf para imagem.
    images = convert_from_path(pdf_path)
    return images

def extrair_informacoes_imagem(image, pdf_path):
    # Converte a imagem PIL em um array NumPy
    image = np.array(image)

    # Converter a imagem para escala de cinza
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Realizar o OCR com pytesseract
    texto = pytesseract.image_to_string(gray, lang='por')

    # Aqui, estou presumindo que você já definiu funções como "padrao_nome", "padrao_cpf", etc.
    # Retornar as informações encontradas (ou None se não encontradas)
    return {
        'NOME_ARQ': os.path.basename(pdf_path),
        'DISTRIBUIDORA': atribuir_distribuidora(os.path.basename(pdf_path)),
        'DECLARADO': buscar_siglas(os.path.basename(pdf_path)),
        'NOME_CPF': padrao_nome(texto),
        'NOME_CNPJ': fantasia_nome(texto),
        'CPF': padrao_cpf(texto),
        'CNPJ': padrao_cnpj(texto),
        'SITU_CAD': padrao_situacao_cadastral(texto),
        'DTA_NASC': padrao_data_nascimento(texto),
        'UF': find_uf(texto),
        'TEXTO': texto  # Adicionando o texto extraído como uma nova coluna
    }

# Lista para armazenar as informações extraídas
informacoes_lista = []

# Converter o PDF em imagens
images = convert_pdf_to_image(pdf_path)

# Loop para processar cada imagem
for image in images:
    # Extrair as informações da imagem
    informacoes = extrair_informacoes_imagem(image, pdf_path)

    # Adicionar as informações à lista
    informacoes_lista.append(informacoes)

# Criar um DataFrame a partir da lista de informações
DF = pd.DataFrame(informacoes_lista)

In [ ]:
# Define o caminho do diretório.
dir_path = '/content/sample_data/exemplos'

# Lista todos os arquivos no diretório.
files = os.listdir(dir_path)

# Filtra apenas os arquivos PDF.
pdf_files = [file for file in files if file.endswith('.pdf') or file.endswith('.PDF')]

def convert_pdf_to_images(pdf_path):
    # Converte pdf para imagens.
    images = convert_from_path(pdf_path)
    return images

def extrair_informacoes_imagem(image):
    # Converte a imagem PIL em um array NumPy
    image = np.array(image)

    # Converter a imagem para escala de cinza
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Realizar o OCR com pytesseract
    texto = pytesseract.image_to_string(gray, lang='por')

    # Retornar as informações encontradas (ou None se não encontradas)
    return {
        'NOME_ARQ': os.path.basename(pdf_path),
        'DISTRIBUIDORA': atribuir_distribuidora(os.path.basename(pdf_path)),
        'DECLARADO': buscar_siglas(os.path.basename(pdf_path)),
        'NOME_CPF': padrao_nome(texto),
        'NOME_CNPJ': fantasia_nome(texto),
        'CPF': padrao_cpf(texto),
        'CNPJ': padrao_cnpj(texto),
        'SITU_CAD': padrao_situacao_cadastral(texto),
        'DTA_NASC': padrao_data_nascimento(texto),
        'UF': find_uf(texto),
        'TEXTO': texto  # Adicionando o texto extraído como uma nova coluna
    }

# Lista para armazenar as informações extraídas
informacoes_lista = []

# Loop para processar cada pdf
for pdf_file in pdf_files:
    # Caminho completo do pdf
    pdf_path = os.path.join(dir_path, pdf_file)

    # Converter o PDF em imagens
    images = convert_pdf_to_images(pdf_path)

    # Loop para processar cada imagem
    for image in images:
        # Extrair as informações da imagem
        informacoes = extrair_informacoes_imagem(image)

        # Adicionar as informações à lista
        informacoes_lista.append(informacoes)

# Criar um DataFrame a partir da lista de informações
DF = pd.DataFrame(informacoes_lista)

# Extrair as KEYWORDS do TEXTO (output do OCR)

In [ ]:
# Função para extrair as palavras-chave

kw_extractor = KeywordExtractor(lan='pt', n=1, top=30)


def extrair_palavras_chave(texto):
    keywords = kw_extractor.extract_keywords(texto)
    return [keyword for keyword, _ in keywords]

# Aplicar a função para extrair as palavras-chave a cada linha do DataFrame
DF['KEYWORDS'] = DF['TEXTO'].apply(extrair_palavras_chave)

# Mensurar a qualidade do OCR através das métricas CER e WER (Character Error Rate & Word Error Rate)

Não é possível mensurar a qualidade do OCR a partir das métricas de CER e WER pois é necessário realizar a comparação com uma referência (texto original).

LINK: https://towardsdatascience.com/evaluating-ocr-output-quality-with-character-error-rate-cer-and-word-error-rate-wer-853175297510

**Verificar se existem as palavras no dicionário**: Cada palavra no texto do OCR é comparada com um dicionário do idioma PT-BR. Se uma grande porcentagem das palavras não estiver no dicionário, isso pode indicar um problema com o OCR.

In [ ]:
spell = SpellChecker(language='pt')

def calculate_quality(text):
    if len(text) == 0:
        return None
    unknown_words = spell.unknown(text)
    return len(unknown_words) / len(text)

DF['COMPARACAO_DICIONARIO'] = DF['KEYWORDS'].apply(calculate_quality)

*   Um valor de 0 indica que todas as palavras da lista são reconhecidas pelo corretor ortográfico, ou seja, todas as palavras existem no dicionário da língua definida (neste caso, português). Isso sugere que as palavras são todas ortograficamente corretas.

*   Um valor de 1 indica que nenhuma palavra na lista é reconhecida pelo corretor ortográfico. Isso pode sugerir que as palavras estão todas ortograficamente incorretas, ou que a lista contém palavras específicas ou técnicas que não estão presentes no dicionário padrão.

In [ ]:
str(DF.KEYWORDS[1])

"['FONTES', 'SANTOS', 'GAMA', 'LOS', 'Poa', 'Tra', 'Sera', 'DEUSDEDIT', 'CAMPINA', 'nen', 'esti', 'enlisafaaesaa', 'coca', 'ADALGISA', 'MARIA', 'GRANDE-PB', 'NASC', 'FERRAZ', 'DIRETOR', 'IACOUELINE', 'OLIVEIRA', 'ASSINATURA', '096995196-57', 'soco']"

## Contagem das palavras mais frequentes para cada tipo e documento (COLUNA:DECLARADO)

Testar as palavras únicas que aparecem somente em cada um dos documentos.

In [ ]:
# Configurar o extrator de palavras-chave
kw_extractor = KeywordExtractor(lan='pt', n=1, top=10)

# Agrupar o DataFrame por tipo de documento
GRUPOS = DF.groupby('DECLARADO')

# Para cada grupo, concatenar todo o texto e extrair as palavras-chave
palavras_chave = {nome: set(palavra for palavra, _ in kw_extractor.extract_keywords(grupo['TEXTO'].str.cat(sep=' ')))
                  for nome, grupo in GRUPOS}

# Obter palavras-chave exclusivas para cada tipo de documento
palavras_chave_exclusivas = {}

for nome in palavras_chave.keys():
    outras_palavras = set().union(*(palavras for outro_nome, palavras in palavras_chave.items() if outro_nome != nome))
    palavras_chave_exclusivas[nome] = palavras_chave[nome] - outras_palavras

# Ver as palavras-chave exclusivas para cada tipo de documento
for tipo_documento, palavras in palavras_chave_exclusivas.items():
    print(f'{tipo_documento}: {palavras}')

CCIR: {'PRIC', 'CADASTRAIS', 'INCLUÍDOS', 'TAXA', 'SERVIÇOS', 'VALOR', 'RURAL', 'IMÓVEL'}
CNPJ: {'Município', 'DATA', 'MONTE', 'JURÍDICA', 'ESPECIAL', 'CÓDIGO', 'INSCRIÇÃO', 'SITUAÇÃO', 'CADASTRAL', 'DESCRIÇÃO'}
CPF: {'Comprovante', 'Cadastral', 'Receita', 'Brasil', 'Inscrição'}
CTRU: {'SANTOS', 'Qtd', 'Dep', 'NAO', 'OQVIJOSSV', 'Rurais', 'Rua', 'Sindicato', 'FRANÇA', 'Trabalhadores'}
IE: {'CEP', 'SERVICOS', 'ALUGUEL', 'Endereço', 'Consulta', 'EQUIPAMENTOS', 'CRC'}
IRR: {'CODEVASF', 'AQUICULTURA', 'unidade', 'MANIÇOBA', 'Documento', 'CONTRATO', 'IRRIGAÇÃO', 'CLÁUSULA', 'consumidora'}
ITR: {'Tipo', 'Nirt', 'DECLARAÇÃO', 'Total', 'SANTANA'}
LIAM: {'Lei', 'AMBIENTAL', 'certificado', 'empreendimento', 'Estadual', 'inema', 'Decreto', 'autorização', 'LICENCIAMENTO'}
NIS: {'Família', 'AMEASNTT', 'ARA', 'QUEIROS', 'FABIANE', 'Buscar', 'Olá', 'Único', 'PROGRAMAS'}
OUTA: {'Art', 'Arts', 'Processo', 'INEMA', 'Declaração', 'qualquer', 'GOVBA', 'uso', 'Outorga'}
PRMA: {'UUUZIYL', 'Usuário', 'SAPCAN

# Paracer sobre o documento

Parecer utilizando verificação das palavras mais frequentes

In [ ]:
# Configurar o extrator de palavras-chave
kw_extractor = KeywordExtractor(lan='pt', n=1, top=10)

# Para cada grupo, concatenar todo o texto e extrair as palavras-chave mais relevantes
palavras_chave = {nome: set([keyword for keyword, _ in kw_extractor.extract_keywords(grupo['TEXTO'].str.cat(sep=' '))])
                  for nome, grupo in grupos}

# Verificar se cada texto contém pelo menos 80% das palavras-chave de seu tipo de documento
def verificar_parecer(row):
    palavras_texto = set(row['KEYWORDS'])
    palavras_relevantes = palavras_chave[row['DECLARADO']]
    intersecao = palavras_texto.intersection(palavras_relevantes)

    return 'IGUAL' if len(intersecao) >= 0.1 * len(palavras_relevantes) else 'DIFERENTE'

# Aplicar a função a cada linha do DataFrame
DF['PARECER'] = DF.apply(verificar_parecer, axis=1)
#df
#df.groupby('PARECER').size()

Parecer utilizando modelo SVM

In [ ]:
# Separar o texto e a coluna "DECLARADO"
textos = DF["TEXTO"]
declarado = DF["DECLARADO"]

# Inicializar o vetorizador TF-IDF
vetorizador = TfidfVectorizer()

# Ajustar o vetorizador aos textos e calcular o TF-IDF
vetores_tfidf = vetorizador.fit_transform(textos)

# Inicializar o classificador SVM
classificador = SVC(probability=True)  # Habilitar o cálculo de probabilidades

# Treinar o classificador com os vetores TF-IDF e as classes correspondentes
classificador.fit(vetores_tfidf, declarado)

# Calcular as previsões do modelo para os textos do DataFrame
previsoes = classificador.predict(vetorizador.transform(textos))

# Calcular as probabilidades do modelo para cada classe
probabilidades = classificador.predict_proba(vetorizador.transform(textos))

# Adicionar as colunas "PARECER" e "PROBABILIDADE" ao DataFrame
DF["PARECER_SVM"] = ["IGUAL" if pred == decl else "DIFERENTE" for pred, decl in zip(previsoes, declarado)]
DF["PROBABILIDADE"] = [probs.max() if pred == decl else 1 - probs.max() for probs, pred, decl in zip(probabilidades, previsoes, declarado)]

# Obter o índice da classe com a maior probabilidade
indice_maior_probabilidade = probabilidades.argmax(axis=1)

# Mapear o índice da classe para o nome do documento
maior_probabilidade_documento = [classificador.classes_[indice] for indice in indice_maior_probabilidade]

# Adicionar a coluna "PARECER_SVM" ao DataFrame
DF["TIPO_SVM"] = maior_probabilidade_documento

# TESTES (NÃO CONSIDERAR! SERÁ DELETADO)

In [ ]:
# Define o caminho do diretório.
dir_path = '/content/sample_data/exemplos'

# Cria a pasta "imagem" se não existir.
image_dir = os.path.join(dir_path, 'imagem')
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

# Lista todos os arquivos no diretório.
files = os.listdir(dir_path)

# Filtra apenas os arquivos PDF.
pdf_files = [file for file in files if file.endswith('.pdf')\
             or file.endswith('.PDF')]

# Função para converter e salvar imagens JPEG.
def convert_pdf_to_jpeg(pdf_path):
    # Converte pdf para jpg.
    images = convert_from_path(pdf_path)

    # Pega o nome do arquivo em formato .pdf sem a extensão.
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]

    # Salva a imagem em formato .jpeg.
    for i, image in enumerate(images):
        image_path = os.path.join(image_dir, f'{base_name}_{i}.jpeg')
        image.save(image_path, format='png')

# Converte e salva todas as imagens.
for pdf_file in pdf_files:
    pdf_path = os.path.join(dir_path, pdf_file)
    convert_pdf_to_jpeg(pdf_path)

In [ ]:
image = cv2.imread('/content/sample_data/exemplos/imagem/1000112647_CTRU_0.jpeg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
texto = pytesseract.image_to_string(gray, lang='por')
print(texto)

In [ ]:
import sys
print(sys.version)

3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
